In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import random
import re, string, unicodedata
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import json
import pickle
from threading import Timer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
words=[]
classes = []
documents = []
ignored_symbols = ['?',',', '!', "'m"]
data_file = open('conversation.json').read()
conversation = json.loads(data_file)

In [ ]:
for intent in conversation['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
all_stopwords = set(stopwords.words('english'))
words = [lemmatizer.lemmatize(w.lower()) for w in words if (w not in ignored_symbols) and (w not in all_stopwords)]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
45/45 [==============================] - 0s 2ms/step - loss: 2.4269 - accuracy: 0.0889
Epoch 2/200
45/45 [==============================] - 0s 325us/step - loss: 2.3119 - accuracy: 0.2222
Epoch 3/200
45/45 [==============================] - 0s 346us/step - loss: 2.3388 - accuracy: 0.1778
Epoch 4/200
45/45 [==============================] - 0s 365us/step - loss: 2.2130 - accuracy: 0.2889
Epoch 5/200
45/45 [==============================] - 0s 387us/step - loss: 2.1284 - accuracy: 0.2667
Epoch 6/200
45/45 [==============================] - 0s 356us/step - loss: 1.9781 - accuracy: 0.3556
Epoch 7/200
45/45 [==============================] - 0s 424us/step - loss: 1.9553 - accuracy: 0.3778
Epoch 8/200
45/45 [==============================] - 0s 316us/step - loss: 1.8490 - accuracy: 0.4889
Epoch 9/200
45/45 [==============================] - 0s 319us/step - loss: 1.7356 - accuracy: 0.4667
Epoch 10/200
45/45 [==============================] - 0s 316us/step - loss: 1.5511 - accuracy

In [ ]:
def clean_up_input(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if (word not in ignored_symbols) and (word not in all_stopwords)]
    return sentence_words

In [ ]:
def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_input(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
def predict_class(sentence, model):
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def time_ran_out():
    print('Is there anything else I can help you?')
    out_of_time=True

In [ ]:
def chatbot_response():
    i=1
    while i>0:
      t=Timer(12,time_ran_out)
      t.start()
      msg=input()
      if msg == "no thanks":
        break 
      ints = predict_class(msg, model)
      res = get_response(ints, conversation)
      print(res)
      t.cancel()
      i=i+1 

In [1]:
#temporary measure: please type "no thanks" to kill the program when testing
chatbot_response()

NameError: ignored